In [1]:
from pathlib import Path

from src.dino.opencv import read_image, write_image
from src.tone_mappers.bronto import bronto
from src.tone_mappers.exposure import exposure_luminance
from src.tone_mappers.linear import linear_luminance
from src.tone_mappers.reinhard import reinhard_luminance


INPUT_IMAGE = Path("images/507.exr")
image = read_image(str(INPUT_IMAGE))
OUTPUT_PATH = Path("_output")

# for pct in [100, 90, 80, 70, 60]:
#     linear_tonemapped = linear_luminance(image, pct=pct)
#     file_path = f"{OUTPUT_PATH / INPUT_IMAGE.stem}_linear_p{pct}.png"
#     write_image(file_path, linear_tonemapped)
#     print("Exported", file_path)

# for key in [0.05, 0.1, 0.18, 0.25, 0.4]:
#     exposure_tonemapped = exposure_luminance(image, key=key)
#     file_path = f"{OUTPUT_PATH / INPUT_IMAGE.stem}_exposure_k={key}.png"
#     write_image(file_path, exposure_tonemapped)
#     print("Exported", file_path)

# reinhard_tonemapped = reinhard_luminance(image)
# file_path = f"{OUTPUT_PATH / INPUT_IMAGE.stem}_reinhard_default.png"
# write_image(file_path, reinhard_tonemapped)
# print("Exported", file_path)

bronto_tonemapped = bronto(image, k=0.18)
file_path = f"{OUTPUT_PATH / INPUT_IMAGE.stem}_bronto_k=0.18.png"
write_image(file_path, bronto_tonemapped)
print("Exported", file_path)

[ WARN:0@399.368] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


Exported _output/507_bronto_k=0.18.png
